## Datatype Support
`redshift_connector` supports Amazon Redshift specific datatypes in order to provide users integration of their data into Python projects. Please see the projects [README](https://github.com/aws/amazon-redshift-python-driver/blob/master/README.rst) for a list of supported datatypes.

### Examples
The following sections provide basic examples showing how to work with Amazon Redshift datatypes.

#### Geometry
- **Send**: A string holding geometry data in WKB (well known binary) format.
- **Receive**: A string holding geometry data in WKB format.

**Note**: Geometry data can be sent and receive in formats other than WKB if Amazon Redshift spatial functions are applied. Please see the [Amazon Redshift documentation for a list of spacial functions](https://docs.aws.amazon.com/redshift/latest/dg/geospatial-functions.html).

[Geometry](https://docs.aws.amazon.com/redshift/latest/dg/GeometryType-function.html)

Sending data in WKB format:

In [ ]:
import redshift_connector

with redshift_connector.connect(...) as conn:
    with conn.cursor() as cursor:
        cursor.execute("create table datatype_test (c1 geometry);")
        cursor.execute(
            "insert into datatype_test (c1) values (%s);",
            (
               '0103000020E61000000100000005000000000000000000000000000000000000000000000000000000000000000000F03F000000000000F03F000000000000F03F000000000000F03F000000000000000000000000000000000000000000000000',
               # using WKB format
            )
        )
        cursor.execute("select c1 from datatype_test;")
        result = cursor.fetchone()
        print("c1={}\n".format(result[0],))

Sending data in WKT (well known text) format:

In [ ]:
import redshift_connector

with redshift_connector.connect(...) as conn:
    with conn.cursor() as cursor:
        cursor.execute("create table datatype_test (c1 geometry);")
        cursor.execute(
            "insert into datatype_test (c1) values (ST_GeomFromText(%s));",
            (
                'LINESTRING(1 2,3 4,5 6,7 8,9 10,11 12,13 14,15 16,17 18,19 20)', # using WKT format
            )
        )
        cursor.execute("select c1, c2 from datatype_test;")
        result = cursor.fetchone()
        print("c1={}\nc2={}".format(result[0], result[1]))

#### Super
- **Send**: A string containing JSON data.
- **Receive**: A string containing JSON data

[Super](https://docs.aws.amazon.com/redshift/latest/dg/r_SUPER_type.html)
[json_parse](https://docs.aws.amazon.com/redshift/latest/dg/JSON_PARSE.html)
[Unnesting SUPER arrays](https://docs.aws.amazon.com/redshift/latest/dg/query-super.html#unnest)
[Querying semistructured data](https://docs.aws.amazon.com/redshift/latest/dg/query-super.html)

In [ ]:
import redshift_connector

with redshift_connector.connect(...) as conn:
    with conn.cursor() as cursor:
        cursor.execute(
            "CREATE TABLE foo AS SELECT json_parse(%s) AS multi_level_array;",
            ('[[1.1, 1.2], [2.1, 2.2], [3.1, 3.2]]',)
        )
        cursor.execute("SELECT array, element FROM foo AS f, f.multi_level_array AS array, array AS element;")
        result = cursor.fetchall()
        print(result)

Retrieving array elements from json array stored in super datatype

In [ ]:
import redshift_connector 
import json

with redshift_connector.connect(...) as conn:
    with conn.cursor() as cursor:
        cursor.execute(
            "CREATE TABLE foo AS SELECT json_parse(%s) AS vals;",
            (json.dumps({"x": [1,2,3,4], "y": [5,6,7,8], "z": [9,10,11,12]}),)
        )
        cursor.execute("SELECT vals.x FROM foo;")
        result = cursor.fetchall()
        print(result)

In [ ]:
import redshift_connector 
import json

with redshift_connector.connect(...) as conn:
    with conn.cursor() as cursor:
        cursor.execute("create table t (s super);")
        cursor.execute("insert into t values (json_parse(%s));", ('[10001,10002,"abc"]',))
        cursor.execute("insert into t values (json_parse(%s));", (json.dumps({"x": [1,2,3,4]}),))
        cursor.execute("select s from t;")
        result = cursor.fetchall()
        print(result)

#### Varbyte
- **Send**: A string or bytes
- **Receive**: A string containing data in hexidecimal format

[Varbyte](https://docs.aws.amazon.com/redshift/latest/dg/r_VARBYTE_type.html)

In [ ]:
import redshift_connector

with redshift_connector.connect(...) as conn:
    with conn.cursor() as cursor:
        cursor.execute("create table t (v varbyte);")
        cursor.execute("insert into t values (%s), (%s);", ('aa', 'abc', ))
        cursor.execute("insert into t values (%s), (%s);", (b'aa', b'abc',))
        cursor.execute("insert into t values (%s), (%s);", (b'\x00\x01\x02\x03',b'\x00\x0a\x0b\x0c'))
        cursor.execute("select v from t;")
        result = cursor.fetchall()
        print(result)